<a href="https://colab.research.google.com/github/tensorflow-project/FineTuning/blob/main/Experiments/CosineSimilarity4emotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/tensorflow-project/FineTuning

Cloning into 'FineTuning'...
remote: Enumerating objects: 1027, done.
remote: Counting objects: 100% (298/298), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 1027 (delta 241), reused 221 (delta 180), pack-reused 729
Receiving objects: 100% (1027/1027), 143.26 MiB | 16.37 MiB/s, done.
Resolving deltas: 100% (747/747), done.


In [ ]:
import sys
import os
from google.colab import drive
import numpy as np

py_file_location = "/content/FineTuning/four_emotions"
sys.path.append(os.path.abspath(py_file_location))
py_file_location = "/content/FineTuning/models"
sys.path.append(os.path.abspath(py_file_location))

drive.mount("/content/drive")

import os
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.use('ps')

import csv
import matplotlib.pyplot as plt
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import textual_inversion_four_emotions as txt

Mounted at /content/drive
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE
202761/202761 [==============================] - 0s 2us/step


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


136824240/136824240 [==============================] - 1s 0us/step


In [ ]:
### embeddings for cosine similarity
sticker_embedding = []
cosine_similarity = []
cosine_similarity.append(txt.cosine_sim(txt.get_embedding("broccoli"), txt.get_embedding(txt.placeholder_token)))

In [ ]:
### EXECUTE THE FOLLOWING TWO BLOCKS ONLY IF YOU WANT TO CONTINUE TRAINING WITH SAVED WEIGTHS
### choose where to load the weights from, either from your google drive or you load our pretrained weights
### make sure to insert the exact name of your weight.npy
drive.mount("/content/drive")
#path = '/content/drive/MyDrive/final10weights_four_emotions.npy'

Mounted at /content/drive


In [ ]:
###load the array of the weights of the text encoder from the training
text_encoder_weights = np.load(path, allow_pickle=True)

### Set the weights of the text encoder
txt.stable_diffusion.text_encoder.set_weights(text_encoder_weights)

In [ ]:
### how long to train for
epochs = 30
epoch_num = list(range(epochs+1))
txt.training(epoch=epochs, model=txt.stable_diffusion, data=txt.train_ds, sticker_embedding=sticker_embedding, cosine_similarity=cosine_similarity)

Epoch 30/30: 100%|██████████| 530/530 [01:44<00:00,  5.07it/s]


In [ ]:
### where to store your newly generated weights
drive.mount("/content/drive")
path = '/content/drive/MyDrive/final30weights_four_emotionsLR4.npy'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
###get the weights of the text encoder and save the to Google Drive
text_encoder_weights = np.array(txt.stable_diffusion.text_encoder.get_weights())

### Save the weights array to a file on your Google Drive
np.save(path, text_encoder_weights)

<ipython-input-7-0443ae8bd6c9>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  text_encoder_weights = np.array(txt.stable_diffusion.text_encoder.get_weights())


In [ ]:


import os
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.use('ps')

In [ ]:
plt.figure()  # Create a new figure
plt.plot(epoch_num, cosine_similarity, 'darkblue')
plt.xlabel('Epoch')
plt.ylabel('Cosine Similarity')
plt.title('Cosine Similarity')
plt.show()



In [ ]:
# Save the plot as an eps file
txt.cosine_plot(epoch_num, cosine_similarity)
plt.savefig('/content/drive/MyDrive/CosineSimilarity_four_emotions30epochsLR4.eps', format='eps')


In [ ]:

# Save the plot as a pdf file
plt.savefig('/content/drive/MyDrive/CosineSimilarity_four_emotions30epochsLR4.pdf', format='pdf')